# Perspective_proj_Rendering

In [1]:
"""
Imports necessary for this notebook
"""
import os 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shdom
from mayavi import mlab # if it fails, use (comandline) jupyter nbextension enable mayavi --user --py 
# mlab.init_notebook()

In [2]:
""" 
 - MiePolydisperse object loads a previously saved mie table.
 - Rayleigh object initilizes Rayleigh scattering according to a temperature profile.
   Temperature profile here is of summer mid-latitude for 0-20km taken from AFGL [1]
 - A Large-Eddy-Simulated [2] cloud michrophysical properties are loaded from a csv file.

[1] G. P. Anderson, S. A. Clough, F. Kneizys, J. H. Chetwynd, and E. P. Shettle. 
    AFGL atmospheric constituentprofiles (0.120 km). Technical report, Air Force Geophysics Lab, 1986.
[2] G. Matheou and D. Chung. Large-eddy simulation of stratified turbulence. 
    part CLARREO: Application of thestretched-vortex model to the atmospheric boundary layer.
    Journal of the Atmospheric Sciences, 71(12):44394460,2014.
"""
# Mie scattering for water droplets
mie = shdom.MiePolydisperse()
mie.read_table(file_path='../mie_tables/polydisperse/Water_672nm.scat')

# Generate a Microphysical medium
droplets = shdom.MicrophysicalScatterer()
droplets.load_from_csv('../synthetic_cloud_fields/jpl_les/rico32x37x26.txt', veff=0.1)

droplets.add_mie(mie)

# Rayleigh scattering for air molecules up to 20 km
df = pd.read_csv('../ancillary_data/AFGL_summer_mid_lat.txt', comment='#', sep=' ')
altitudes = df['Altitude(km)'].to_numpy(dtype=np.float32)
temperatures = df['Temperature(k)'].to_numpy(dtype=np.float32)
temperature_profile = shdom.GridData(shdom.Grid(z=altitudes), temperatures)
air_grid = shdom.Grid(z=np.linspace(0, 20, 20))
rayleigh = shdom.Rayleigh(wavelength=0.672)
rayleigh.set_profile(temperature_profile.resample(air_grid))
air = rayleigh.get_scatterer()

Reading mie table from file: ../mie_tables/polydisperse/Water_672nm.scat


Vadim add visualization of 3d fields

In [3]:
# use mlab.init_notebook() if you want to disply mlab here.
Grid_bounding_box = droplets.bounding_box
Grid_shape = droplets.grid.shape
xgrid = np.linspace(Grid_bounding_box.xmin, Grid_bounding_box.xmax,Grid_shape[0])
ygrid = np.linspace(Grid_bounding_box.ymin, Grid_bounding_box.ymax,Grid_shape[1])
zgrid = np.linspace(Grid_bounding_box.zmin, Grid_bounding_box.zmax,Grid_shape[2])   
X, Y, Z = np.meshgrid(xgrid, ygrid, zgrid, indexing='ij')
LWC_MAT = droplets.lwc.data;
scf = mlab.pipeline.scalar_field(X, Y, Z,LWC_MAT)
figh = mlab.gcf()
mlab.pipeline.volume(scf, figure=figh,vmin=0.0,vmax=1.24) # no working on servers
isosurface = mlab.pipeline.iso_surface(scf, contours=[0.0001],color = (1, 1,1),opacity=0.1,transparent=True)
mlab.pipeline.volume(isosurface, figure=figh)
mlab.pipeline.image_plane_widget(scf,plane_orientation='z_axes',slice_index=10)
                            
# figh.scene.anti_aliasing_frames = 0
mlab.scalarbar()
mlab.axes(figure=figh, xlabel="x (km)", ylabel="y (km)", zlabel="z (km)") 
mlab.show()

initilize an RteSolver object

In [4]:
"""
Generate an Medium with two type of scatterers and initilize an RteSolver object. 
This will initialize all of shdom internal structures and grids.

SceneParameters() contains parameters such as surface albedo and solar radiance
NumericalParameters() contains parameters such as number of angular bins and split cell ratio.
All parameters have default values which can be viewed using print(params.info).
"""
atmospheric_grid = droplets.grid + air.grid # Add two grids by finding the common grid which maintains the higher resolution grid.
atmosphere = shdom.Medium(atmospheric_grid)
atmosphere.add_scatterer(droplets, name='cloud')
atmosphere.add_scatterer(air, name='air')

numerical_params = shdom.NumericalParameters()
scene_params = shdom.SceneParameters(
    wavelength=mie.wavelength,
    source=shdom.SolarSource(azimuth=65, zenith=135)
)

rte_solver = shdom.RteSolver(scene_params, numerical_params)
rte_solver.set_medium(atmosphere)

print(rte_solver.info)

Scene Parameters: 
   Wavelength: [0.672 micron]
   Surface: [Lambertian, albedo: 0.05]
   Source: [Solar, flux: 1.0, azimuth: 65deg, zenith: 135deg]
   Boundary Conditions: [x:open, y:open]

Numerical Parameters: 
   num_mu_bins: 8
   num_phi_bins: 16
   split_accuracy: 0.1
   deltam: True
   spherical_harmonics_accuracy: 0.01
   solution_accuracy: 0.0001
   acceleration_flag: True
   max_total_mb: 10000.0
   adapt_grid_factor: 5
   num_sh_term_factor: 5
   cell_to_point_ratio: 1.5
   high_order_radiance: True



Solve the Radiative Transfer

In [5]:
"""
Solve the Radiative Transfer for the domain using SHDOM: SOLVE_RTE procedure (src/unpolarized/shdomsub1.f).
The outputs are the source function (J) and radiance field (I) which are stored in 
the shdom.RteSolver object. These are subsequently used for the rendering of an image.
"""
rte_solver.solve(maxiter=100)